# **Course**: Deep Learning

[<img align="right" width="400" height="100" src="https://www.tu-braunschweig.de/typo3conf/ext/tu_braunschweig/Resources/Public/Images/Logos/tu_braunschweig_logo.svg">](https://www.tu-braunschweig.de/en/) 

[Mehdi Maboudi](https://www.tu-braunschweig.de/en/igp/staff/mehdi-maboudi) \([m.maboudi@tu-bs.de](m.maboudi@tu-bs.de)) and [Pedro Achanccaray](https://www.tu-braunschweig.de/en/igp/staff/pedro-diaz) (p.diaz@tu-bs.de)

[Technical University of Braunschweig](https://www.tu-braunschweig.de/en/)  
[Institute of Geodesy and Photogrammetry](https://www.tu-braunschweig.de/igp) 

# **Assignment 05:** Transfer learning and Fine tuning

In this assignment you will explore the differences between transfer learning and fine tuning.

For this, you will use the **VGG16** pre-trained network with **ImageNet** dataset.

<center>
<img width=600 src="https://miro.medium.com/max/1400/1*NNifzsJ7tD2kAfBXt3AzEg.png" img>

</center>

- **Transfer learning:** 
  1. Take a pre-trained model as $base\_model$
  2. Freeze the $base\_model$
  3. Add a $head$ (classification layers) to the $base\_model$
  4. Train the new model
  5. $trainable\_parameters = head\_parameters $

- **Fine tuning:**
  1. Take a pre-trained model as $base\_model$
  2. Freeze some layers of the $base\_model$
  3. Add a $head$ (classification layers) to the $base\_model$
  4. Train the new model
  5. $trainable\_parameters = head\_parameters + base\_model\_parameters $

## **PyTorch**

For PyTorch, you can visit the following links:
- [VGG16 PyTorch documentation](https://pytorch.org/vision/main/models/generated/torchvision.models.vgg16.html)
- [Transfer learning with PyTorch](https://debuggercafe.com/transfer-learning-with-pytorch/)
- [Freeze layers of PyTorch model](https://medium.com/@shuklaatul032/freeze-layers-of-pytorch-model-48d2725223b3)


## **Load packages**

In [3]:


# Management of files
import os
from os.path import exists, join

# Tensorflow and Keras
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, \
                                       EarlyStopping

# Monitor training
import wandb
from wandb.integration.keras import WandbMetricsLogger

# Working with arrays
import numpy as np

import matplotlib.pyplot as plt

# External files with functions to load the dataset,
# create a CNN model, and a data generator.
from importlib import reload
import datasets
import data_generator
# Useful to reload modified external files without need
# of restarting the kernel. Just run again this cell.
reload(datasets)
reload(data_generator)

from datasets import *
from models import *
from data_generator import *

## **Functions**

In [4]:
def freeze_up_to(model, freeze_layer_name):
  """Function to freeze some layers of the model

  Args:
      model (keras.Model): a keras.Model
      freeze_layer_name (str): layer name of "model". All layers up
        to this layer will be freezed.

  Returns:
      keras.Model: a keras.Model with some layers freezed.
  """
  # Getting layer number based on layer name
  for id_layer, layer in enumerate(model.layers):
    if layer.name == freeze_layer_name:
      layer_number = id_layer
      break

  # Froze layers
  for layer in model.layers[:layer_number]:
    layer.trainable = False

  return model

## **Base models**

### **VGG16** model with top layers (classification layers)

In [44]:
vgg16_full = VGG16(include_top=True,
                   weights="imagenet",      
                   input_shape=(224, 224, 3))

vgg16_full.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_13 (InputLayer)       [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

### **VGG16** model without top layers (classification layers): feature extractor, and its original input shape.

In [47]:
vgg16_base = VGG16(include_top=False,
                   weights="imagenet",      
                   input_shape=(64,64,3))

vgg16_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_16 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

### **VGG16** model without top layers (classification layers): feature extractor, and a different input shape.

In [46]:
vgg16_dif_input = VGG16(include_top=False,
                        weights="imagenet",      
                        input_shape=(64,64,3))

vgg16_dif_input.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_15 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

**Variables**

In [8]:
PROJECT_DIR = "." # os.getcwd()
SEED = 42
BATCH_SIZE = 32
TARGET_SIZE = 64

**Download the dataset**

In [9]:
url_dataset = "https://zenodo.org/record/7711810/files/EuroSAT_RGB.zip?download=1"
filename = "EuroSAT_RGB.zip"

if not exists("EuroSAT_RGB"):
  !pip install wget
  import wget
  f = wget.download(url_dataset, PROJECT_DIR)
  import zipfile
  with zipfile.ZipFile(filename, "r") as zip_ref:
    zip_ref.extractall(".")
  os.remove(join(PROJECT_DIR, filename))

**Reading the Dataset**

In [10]:
path_data = join(PROJECT_DIR, "EuroSAT_RGB")

df, n_classes = read_eurosat(path_data=path_data, SEED=SEED)
classes = np.unique(df["class_str"].values)

df

,path_image,class_str,class_int
0,.\EuroSAT_RGB\Forest\Forest_2313.jpg,Forest,1
1,.\EuroSAT_RGB\PermanentCrop\PermanentCrop_2358...,PermanentCrop,6
2,.\EuroSAT_RGB\HerbaceousVegetation\HerbaceousV...,HerbaceousVegetation,2
3,.\EuroSAT_RGB\Pasture\Pasture_1415.jpg,Pasture,5
4,.\EuroSAT_RGB\Highway\Highway_1611.jpg,Highway,3
...,...,...,...
26995,.\EuroSAT_RGB\River\River_76.jpg,River,8
26996,.\EuroSAT_RGB\Forest\Forest_2391.jpg,Forest,1
26997,.\EuroSAT_RGB\AnnualCrop\AnnualCrop_861.jpg,AnnualCrop,0
26998,.\EuroSAT_RGB\Pasture\Pasture_1796.jpg,Pasture,5


### **Train, Validation and Test sets**

In [14]:
splits = train_val_test_split(df,
                              val_size=0.2,
                              test_size=0.2,
                              SEED=SEED,
                             )

x_train = splits["x_train"]
y_train = splits["y_train"]
x_val = splits["x_val"]
y_val = splits["y_val"]
x_test = splits["x_test"]
y_test = splits["y_test"]

#SANITY CHECK

# Number of samples per class
_, counts_train = np.unique(y_train, return_counts=True)
_, counts_val = np.unique(y_val, return_counts=True)
_, counts_test = np.unique(y_test, return_counts=True)

print("Samples per class - train: {}".format(counts_train))
print("Samples per class - val: {}".format(counts_val))
print("Samples per class - test: {}".format(counts_test))

Samples per class - train: [1800 1800 1800 1500 1500 1200 1500 1800 1500 1800]
Samples per class - val: [600 600 600 500 500 400 500 600 500 600]
Samples per class - test: [600 600 600 500 500 400 500 600 500 600]


**Data generator**

In [15]:
data_gen_train = DataGenerator(path_images=x_train,
                               labels=y_train,
                               batch_size=BATCH_SIZE,
                               n_classes=n_classes,
                               target_size=TARGET_SIZE,
                               shuffle=True)

data_gen_val = DataGenerator(path_images=x_val,
                             labels=y_val,
                             batch_size=BATCH_SIZE,
                             n_classes=n_classes,
                             target_size=TARGET_SIZE,
                             shuffle=False)

# For sanity check, let's see the generator's output
for i, (x, y) in enumerate(data_gen_train):
    print(i, x.shape, y.shape)

0 (32, 64, 64, 3) (32, 10)
1 (32, 64, 64, 3) (32, 10)
2 (32, 64, 64, 3) (32, 10)
3 (32, 64, 64, 3) (32, 10)
4 (32, 64, 64, 3) (32, 10)
5 (32, 64, 64, 3) (32, 10)
6 (32, 64, 64, 3) (32, 10)
7 (32, 64, 64, 3) (32, 10)
8 (32, 64, 64, 3) (32, 10)
9 (32, 64, 64, 3) (32, 10)
10 (32, 64, 64, 3) (32, 10)
11 (32, 64, 64, 3) (32, 10)
12 (32, 64, 64, 3) (32, 10)
13 (32, 64, 64, 3) (32, 10)
14 (32, 64, 64, 3) (32, 10)
15 (32, 64, 64, 3) (32, 10)
16 (32, 64, 64, 3) (32, 10)
17 (32, 64, 64, 3) (32, 10)
18 (32, 64, 64, 3) (32, 10)
19 (32, 64, 64, 3) (32, 10)
20 (32, 64, 64, 3) (32, 10)
21 (32, 64, 64, 3) (32, 10)
22 (32, 64, 64, 3) (32, 10)
23 (32, 64, 64, 3) (32, 10)
24 (32, 64, 64, 3) (32, 10)
25 (32, 64, 64, 3) (32, 10)
26 (32, 64, 64, 3) (32, 10)
27 (32, 64, 64, 3) (32, 10)
28 (32, 64, 64, 3) (32, 10)
29 (32, 64, 64, 3) (32, 10)
30 (32, 64, 64, 3) (32, 10)
31 (32, 64, 64, 3) (32, 10)
32 (32, 64, 64, 3) (32, 10)
33 (32, 64, 64, 3) (32, 10)
34 (32, 64, 64, 3) (32, 10)
35 (32, 64, 64, 3) (32, 10)
36

## **Transfer learning**

In [48]:
n_classes = 10 # For EuroSAT

**1.** Take a pre-trained model as $base\_model$


In [81]:
vgg16_base = VGG16(include_top=False,
                   weights="imagenet",      
                   input_shape=(64,64,3))

 **2.** Freeze the $base\_model$
 

In [82]:
vgg16_base.trainable = False

 **3.** Add a $head$ (classification layers) to the $base\_model$
  

In [83]:
input = Input(shape=(64,64,3))
x = preprocess_input(input)
x = vgg16_base(x)
x = Flatten()(vgg16_base.output)
x = Dense(100, activation="relu")(x)
output = Dense(n_classes, activation="softmax")(x)

model = Model(inputs = vgg16_base.inputs, outputs=output)

**4.** Train the new model: `model.fit(...)`

In [84]:
# TODO: Set up the callbacks to be executed during the model training.
#       Remember to use Wandb (or another tool) to visualize the model training
#       and to share your report.

# Callbacks
cb_autosave = ModelCheckpoint("cnn_eurosat_rgb_model_transfer_learning.keras",
                              mode="max",
                              save_best_only=True,
                              monitor="val_accuracy",
                              verbose=1)

cb_early_stop = EarlyStopping(patience=10,
                              verbose=1,
                              mode="auto",
                              monitor="val_accuracy")

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="CNN for image classification",
    name="cnn-classification-euroSAT_RGB_Transfer_Learning",

    # track hyperparameters and run metadata
    config={
    "architecture": "CNN",
    "dataset": "EuroSAT_RGB",
    "bs": BATCH_SIZE
    }
)

cb_wandb = WandbMetricsLogger()

callbacks = [cb_autosave, cb_early_stop, cb_wandb]

In [85]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data_gen_train,
          epochs=40,
          validation_data=data_gen_val,
          callbacks=callbacks
                    )



Epoch 1/40
507/507 [==============================] - ETA: 0s - loss: 0.7058 - accuracy: 0.7590
Epoch 1: val_accuracy improved from -inf to 0.82704, saving model to cnn_eurosat_rgb_model_transfer_learning.keras
507/507 [==============================] - 379s 747ms/step - loss: 0.7058 - accuracy: 0.7590 - val_loss: 0.5121 - val_accuracy: 0.8270
Epoch 2/40
507/507 [==============================] - ETA: 0s - loss: 0.4556 - accuracy: 0.8435
Epoch 2: val_accuracy improved from 0.82704 to 0.83315, saving model to cnn_eurosat_rgb_model_transfer_learning.keras
507/507 [==============================] - 367s 725ms/step - loss: 0.4556 - accuracy: 0.8435 - val_loss: 0.4649 - val_accuracy: 0.8331
Epoch 3/40
507/507 [==============================] - ETA: 0s - loss: 0.3825 - accuracy: 0.8655
Epoch 3: val_accuracy did not improve from 0.83315
507/507 [==============================] - 365s 721ms/step - loss: 0.3825 - accuracy: 0.8655 - val_loss: 0.4719 - val_accuracy: 0.8331
Epoch 4/40
507/507 [===

**5.** $trainable\_parameters = head\_parameters $

In [53]:
model.summary()

Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_18 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 tf.__operators__.getitem_7  (None, 64, 64, 3)         0         
  (SlicingOpLambda)                                              
                                                                 
 tf.nn.bias_add_7 (TFOpLamb  (None, 64, 64, 3)         0         
 da)                                                             
                                                                 
 vgg16 (Functional)          (None, 2, 2, 512)         14714688  
                                                                 
 dense_10 (Dense)            (None, 2, 2, 100)         51300     
                                                                 
 dense_11 (Dense)            (None, 2, 2, 10)          1010

In [86]:
data_gen_test = DataGenerator(path_images=x_test,
                              labels=y_test,
                              batch_size=BATCH_SIZE,
                              n_classes=n_classes,
                              target_size=TARGET_SIZE,
                              shuffle=False)

print("Train:")
scores_train = model.evaluate(data_gen_train)
print("Validation:")
scores_val = model.evaluate(data_gen_val)
print("Test:")
scores_test = model.evaluate(data_gen_test)

Train:
507/507 [==============================] - 259s 511ms/step - loss: 0.0882 - accuracy: 0.9734
Validation:
169/169 [==============================] - 89s 525ms/step - loss: 0.4862 - accuracy: 0.8746
Test:
169/169 [==============================] - 89s 525ms/step - loss: 0.4523 - accuracy: 0.8835


## **Fine tuning**

  **1.** Take a pre-trained model as $base\_model$
  

In [68]:
vgg16_base = VGG16(include_top=False,
                   weights="imagenet",      
                   input_shape=(64,64,3))

**2.** Freeze some layers of the $base\_model$
  

In [69]:
vgg16_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

In [70]:
vgg16_base = freeze_up_to(vgg16_base, "block5_conv2")
vgg16_base.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0     

**3.** Add a $head$ (classification layers) to the $base\_model$
  

In [72]:
#input = Input(shape=(64,64,3))
x = preprocess_input(input)
x = vgg16_base(x)
x = Flatten()(vgg16_base.output)
x = Dense(100, activation="relu")(x)
output = Dense(n_classes, activation="softmax")(x)

model = Model(inputs = vgg16_base.inputs,outputs=output)

In [73]:
# TODO: Set up the callbacks to be executed during the model training.
#       Remember to use Wandb (or another tool) to visualize the model training
#       and to share your report.

# Callbacks
cb_autosave = ModelCheckpoint("cnn_eurosat_rgb_model_Fine_Tuning.keras",
                              mode="max",
                              save_best_only=True,
                              monitor="val_accuracy",
                              verbose=1)

cb_early_stop = EarlyStopping(patience=10,
                              verbose=1,
                              mode="auto",
                              monitor="val_accuracy")

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="CNN for image classification",
    name="cnn-classification-euroSAT_RGB_Fine_Tuning",

    # track hyperparameters and run metadata
    config={
    "architecture": "CNN",
    "dataset": "EuroSAT_RGB",
    "bs": BATCH_SIZE
    }
)

cb_wandb = WandbMetricsLogger()

callbacks = [cb_autosave, cb_early_stop, cb_wandb]

epoch/accuracy,▁▂▃▃▄▅▆▆▆▇██
epoch/epoch,▁▂▂▃▄▄▅▅▆▇▇█
epoch/learning_rate,▁▁▁▁▁▁▁▁▁▁▁▁
epoch/loss,█▇▆▆▅▄▃▃▃▂▁▁
epoch/val_accuracy,▃▃▃▄▁▆█▅▇▇█▅
epoch/val_loss,▃▂▃▁▆▂▁▃▅▄▅█
epoch/accuracy,0.9529
epoch/epoch,11
epoch/learning_rate,0.001
epoch/loss,0.14344
epoch/val_accuracy,0.86815


**4.** Train the new model: `model.fit(...)`


In [75]:
model.compile(optimizer=Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(data_gen_train,
          epochs=40,
          validation_data=data_gen_val,
          callbacks=callbacks
                    )

Epoch 1/40
507/507 [==============================] - ETA: 0s - loss: 0.6496 - accuracy: 0.7722
Epoch 1: val_accuracy improved from -inf to 0.84352, saving model to cnn_eurosat_rgb_model_Fine_Tuning.keras
507/507 [==============================] - 461s 909ms/step - loss: 0.6496 - accuracy: 0.7722 - val_loss: 0.4584 - val_accuracy: 0.8435
Epoch 2/40
507/507 [==============================] - ETA: 0s - loss: 0.3896 - accuracy: 0.8710
Epoch 2: val_accuracy improved from 0.84352 to 0.86463, saving model to cnn_eurosat_rgb_model_Fine_Tuning.keras
507/507 [==============================] - 461s 910ms/step - loss: 0.3896 - accuracy: 0.8710 - val_loss: 0.4077 - val_accuracy: 0.8646
Epoch 3/40
507/507 [==============================] - ETA: 0s - loss: 0.3105 - accuracy: 0.8980
Epoch 3: val_accuracy improved from 0.86463 to 0.87667, saving model to cnn_eurosat_rgb_model_Fine_Tuning.keras
507/507 [==============================] - 468s 924ms/step - loss: 0.3105 - accuracy: 0.8980 - val_loss: 0.37

  **5.** $trainable\_parameters = head\_parameters + base\_model\_parameters $

In [74]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_24 (InputLayer)       [(None, 64, 64, 3)]       0         
                                                                 
 block1_conv1 (Conv2D)       (None, 64, 64, 64)        1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 64, 64, 64)        36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 32, 32, 64)        0         
                                                                 
 block2_conv1 (Conv2D)       (None, 32, 32, 128)       73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 32, 32, 128)       147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 16, 16, 128)       0  

**Testing the model**

In [76]:
data_gen_test = DataGenerator(path_images=x_test,
                              labels=y_test,
                              batch_size=BATCH_SIZE,
                              n_classes=n_classes,
                              target_size=TARGET_SIZE,
                              shuffle=False)

print("Train:")
scores_train = model.evaluate(data_gen_train)
print("Validation:")
scores_val = model.evaluate(data_gen_val)
print("Test:")
scores_test = model.evaluate(data_gen_test)

Train:
507/507 [==============================] - 299s 590ms/step - loss: 0.0613 - accuracy: 0.9821
Validation:
169/169 [==============================] - 113s 668ms/step - loss: 0.5643 - accuracy: 0.9033
Test:
169/169 [==============================] - 114s 677ms/step - loss: 0.5319 - accuracy: 0.9076


**Comments**

It can be seen that for this task Fine tuning performing better than Transfer
Learning, which is expected as in Fine Tuning we don't freeze all the layers of the feature extractor hence the model training or getting adpated better for EuroSAT_RGB Dataset.